# TODO
- vP is not allow to be 0 or 1
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- camelCase to snake_case if required
- gen_prime() is not used

### Import and Configuration

In [398]:
import sympy
import random
import coloredlogs, logging

In [399]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [400]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 24

MAX_ITERS = 10

msg = 12345678900987654321

In [401]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))



# Voraussetzungen

In [402]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

attacker = {}

created = False
while not created:
    try:
        attacker = create_attacker_key()
        created = True
    except ValueError:
            logger.error("Trying to generate Key failed. Trying again ...")
logger.info("Attacker Key successfully generated!")

ERROR Trying to generate Key failed. Trying again ...
INFO Attacker Key successfully generated!


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [403]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
user = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, attacker["E"] ,attacker["N"])

    # Schritt 2
    t = gen_random_number_with_bit_limit() 
    tempN = concatenate_in_binary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(gen_prime_with_boundaries(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    user["P"] = P
    user["Q"] = Q

attacker["vP"] = vP
attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Bounds: [235332384353852119118153983504917858811211438818636385302821176047498775983399750001889850476277064987839056749936456646583461861235632862472426871561031676397641244065648965011860607358811912215264801415235688422196844422531465096994511304789429020119443929784781578315711106114347943068850257984637899311120656155, 235332384353852119118153983504917858811211438818636385302821176047498775983399750001889850476277064987839056749936456646583461861235632862472426871561031676397641244065648965011860607358811912215264801415235688422196844422531465096994511304789429020119443929784781578315711106114347943068850257984637899311178795965] 	 P: 3092017579620348678391774061308837021163290015747337833361542382780322839703336564248139785750439702118628932182294218539386847060323882247074412881806496867331855186679184469821650989951455616296730398981084770050762298477940238038948481575605099723322245147716022080647690236605669274130777353339149 	 deltaB: 58139810
INFO Amount of failed atte

## Schritt 4

In [404]:
user["N"] = calc_n(user["P"], user["Q"])

## Schritt 5

In [405]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        user.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [406]:
cipher = pow(msg, user["E"], user["N"])
signature = pow(msg, user["D"], user["N"])

# Angriff

## Schritt 1

In [407]:
pub_user = {
    "N" : user["N"],
    "E" : user["E"]
}

collection_user = {
    "N" : pub_user["N"],
    "E" : pub_user["E"]
}

## Schritt 2

In [408]:
collection_user["vP"] = split_in_binary(user["N"])[0]
collection_user["vP'"] = collection_user["vP"]+1

### Validation test for vP

In [409]:
testVar = "vP"
altTestVar = "vP'"

if(collection_user[testVar] == attacker[testVar] or collection_user[altTestVar] == attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(attacker[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 4047697882162818645717259269833761684653622688764598007378037313230087624552858416281378015785909811420817374568443159089536635601098076824687599082494210667999461786930382588988276753352386465877344764800196492375903100661033337162017408305590310866772692923049585053036575869787732905874188005553683302114
INFO Current vP: 4047697882162818645717259269833761684653622688764598007378037313230087624552858416281378015785909811420817374568443159089536635601098076824687599082494210667999461786930382588988276753352386465877344764800196492375903100661033337162017408305590310866772692923049585053036575869787732905874188005553683302113 	 Current vP': 404769788216281864571725926983376168465362268876459800737803731323008762455285841628137801578590981142081737456844315908953663560109807682468759908249421066799946178693038258898827675335238646587734476480019649237590310066103333716201740830559031086677269292304958505303657586978773

## Schritt 3

In [410]:
collection_user["P"] = pow(collection_user["vP"], attacker["D"], attacker["N"])
collection_user["P'"] = pow(collection_user["vP'"], attacker["D"], attacker["N"])

### Validation test for P and P'

In [411]:
testVar = "P"
altTestVar = "P'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state: {2}".format(testVar, altTestVar, user["P"]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} or {1} doesn't have the expected value.".format(testVar, altTestVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(user[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables P or P' are in the expected state: 3092017579620348678391774061308837021163290015747337833361542382780322839703336564248139785750439702118628932182294218539386847060323882247074412881806496867331855186679184469821650989951455616296730398981084770050762298477940238038948481575605099723322245147716022080647690236605669274130777353339149
INFO Current P: 4406827278648315635830899346536241467756303154982469177055382070165684283215878346182816537212117227861481739565913197027225800604528461732969775096947980154071887213706756835618958779226840234829511751744973808151077231285783478658594984189917477484592072779095762707658189083922483379767225531368876539343 	 Current P': 30920175796203486783917740613088370211632900157473378333615423827803228397033365642481397857504397021186289321822942185393868470603238822470744128818064968673318551866791844698216509899514556162967303989810847700507622984779402380389484815756050997233222451477160220806476902366056692741307

# Schritt 4

In [412]:
# Potential to optimize when N/P not hole Number
collection_user["Q"] = int(pub_user["N"]//collection_user["P"])
collection_user["Q'"] = int(pub_user["N"]//collection_user["P'"])

### Validation test for Q and Q'

In [413]:
testVar = "Q"
altTestVar = "Q'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(attacker[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables Q or Q' are in the expected state!
INFO Current Q: 165119216948133375916895185418540640508912112206718825801385121007435126859002344251586735463487691128329510538899915890902884474349664632565034765465490168001864491605670769989646263022585049379604078105207344683813782635337235265753757931598359803648307240394077239956764441266210788667169003162958846549133 	 Current Q': 235332384353852119118153983504917858811211438818636385302821176047498775983399750001889850476277064987839056749936456646583461861235632862472426871561031676397641244065648965011860607358811912215264801415235688422196844422531465096994511304789429020119443929784781578315711106114347943068850257984637899311155699687


## Schritt 5

In [414]:
collection_user["PhiN"] = int(calc_phi_n(collection_user["P"], collection_user["Q"]))
collection_user["PhiN'"] = int(calc_phi_n(collection_user["P'"], collection_user["Q'"]))

collection_user["D"] = modular_multiplicative_inverse(pub_user["E"], collection_user["PhiN"])
collection_user["D'"] = modular_multiplicative_inverse(pub_user["E"], collection_user["PhiN'"])

### Validation test for PhiN and PhiN'

In [415]:
testVar = "PhiN"
altTestVar = "PhiN'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(attacker[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables PhiN or PhiN' are in the expected state!
INFO Current PhiN: 727651869476083442296431265304328260130488463356267942199192915725920381058258280437224156741528056268170682267380706792846837842587970283816994390662005689172906515890985064895024637656199314538849907530656133776926326632045569074395804390781433020619692348863314470222855597570744749626984004974221161444641429987330905936079752050969474773559197474331752824489205149854914693309865592476947836129660772491701439773714711085743314274457058458723237796135885135745024296263675116395973646778395219942469969515704124296725134971083771276876172684074843789867869887852517489922442928033371990837427685299831133951144 	 Current PhiN': 72765186947608344229643126530432826013048846335626794219919291572592038105825828043722415674152805626817068226738070679284683784258797028381699439066200568917290651589098506489502463765619931453884990753065613377692632663204556907439580439078143302061969234886331447022285

### Validation test for D and D'

In [416]:
testVar = "D"
altTestVar = "D'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(attacker[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables D or D' are in the expected state!
INFO Current D: 585934237268433884680555520757821006025089150810997270452996135642522487596128358538133122109963228610405899191242840526403499907943527408759544150302974292914259053101529743956927158750486083070002077759362133570498085590627291709463200712799007656700230775387394823337515578232157600623538259220052388008576257453368538367101587728852588495430665552480424673188715128451003450112784181634587609382037443683790226607538435511664127503637980774863076250293407789014940677582479010598054797464581090712482532023929288058822887038299035073234756871029826377812489091530599592530587006440715017050880742730487944703049 	 Current D': 48979407922346071644208262260456134500200448712175418469223791574733740772394228883238028061241388336686271079761405769961465133570236234646539955368209895131259590829630460888931621626814740622193302746246188695599932821224740580355976799204925669883603320685652767962236678259968924

## Schritt 6

In [417]:
collection_user["Sig"] = pow(msg, collection_user["D"], user["N"])
collection_user["Sig'"] = pow(msg, collection_user["D'"], user["N"])

### Validation test for signature and signature'

In [418]:
testVar = "Sig"
altTestVar = "Sig'"

if(collection_user[testVar] == signature):
    collection_user["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_user[testVar]))
elif(collection_user[altTestVar] == signature):
    collection_user["correct_key"] = "D'"
    logger.info("D' is the right privat Key: {0}".format(collection_user[altTestVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D' is the right privat Key: 86066963736711685622509521424941407911074981372191145770647584172282481287311713735461081040075753046696369841989035808812121110978008006498436099819592032224751835474269281600755435116120465597789091544136010130047102909339573720516925126247873409194418918048496771749472009620647699614409831934474351107277884652518941346489901646107212178953750679633090339780256339228556405019105255251733759817073965189870106897014859153632488287842475565229578620886661251504115570054774943735180381146551188562047933688380079018077337571006274385000751880352795925751685038664588163122948274223572953370549514360706794729968


# Result

In [419]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [420]:
logger.info(sort_dic(attacker))
logger.info(sort_dic(user))
logger.info(sort_dic(collection_user))



INFO {'D': 162938215751245927518218999433311869922928638796942106678059356391104564478606757989153583300394686238253915499519051895310511390810659204010836150144904555702399490377398501631934978205777496037773112034670036578662845227568705070042638865545843390522434470922226955361959698150952695312732162679455272193, 'E': 65537, 'N': 11734595434823521265671998424023032988064806814104609720175797845938263562895001206961712515118644562633458088013164949519741741779734255223360625024227055689392354465655986088607638222043540675526808279995620165205676581501937565848549101708394189364258056074498139312543249121118889325740519752296012660059, 'P': 10792094852350046508808938806440892214219890864160348565800832008684121179225868529767781514213454734082254194520322828454545593084724237631640338683137473, 'PhiN': 1173459543482352126567199842402303298806480681410460972017579784593826356289500120696171251511864456263345808801316494951974174177973425522336062502422704380996500593501490725434299194